<a href="https://colab.research.google.com/github/abhinaya-03/WebScraping-of-Metacritic-Movies/blob/main/DAC_Metacritic_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [261]:
# This project is for webscraping the name, date published and the summary of a specific year of the movies on this side for the DAC first practice project (Extras: rating + score)

In [262]:
# Add the website you are trying to get the data from
url = "https://www.metacritic.com/browse/movie/all/all/2020/metascore/?page=1"

import requests

# We have to get the html from the website
# We are using headers because the website is blocking us

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
#print(response.status_code)  # 200 means success
#print(response.text)  # Print first 1000 characters to check if content is fetched


In [263]:
# Implementing BeautifulSoup

from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')
productListings = soup.find_all(class_='c-productListings')
productListings;

In [264]:
# making variables and finding the elements through inspecting the page

title_element = soup.find_all(class_='c-finderProductCard_titleHeading')
date_element = soup.find_all(class_='u-text-uppercase')
summary_element = soup.find_all(class_='c-finderProductCard_description')
rating_element = soup.find_all(class_='c-finderProductCard_meta')
score_element = soup.find_all(class_='c-finderProductCard_score')


# for each loop for finding each of the items in a group, there are date, rating and score. We are trying to only get the rating element.
#for item in rating_element:
#  print(item.text)

In [265]:
titles = []

for title in title_element:
  titles.append(title.text.split(" ", 1)[1])

titles;

In [266]:
dates = []

for date in date_element:
  dates.append(date.text[11:].split('\n')[0])

dates;

In [267]:
summaries = []

for summary in summary_element:
  summaries.append(summary.text)

summaries;

In [268]:
ratings = [] #array
count = 0 #counting becuase of odd and even indexes

  # item        object
for rating in rating_element:
  if count % 2 == 0: #we want the even indexes bc they are the ratings
    try:
      ratings.append((rating.text.split('\n')[4])[15:]) # try appending/adding to the array by splitting it by \n in the middle + parsing
    except:
      ratings.append('Not Rated') #if it is empty then it would be Not Rated
  count += 1

ratings;

In [269]:
scores = []

for score in score_element:
  scores.append(score.text[:2])

scores;

In [270]:
import pandas as pd

data = {
    'Title': titles,
    'Date': dates,
    'Summary': summaries,
    'Rating': ratings,
    'Metascore': scores
}

df = pd.DataFrame(data)
df;

In [271]:
pagenum = soup.find_all("c-navigationPagination_itemButtonContent")

pagenum;

In [272]:
import requests
from bs4 import BeautifulSoup
import pandas as pd #basically excel

url = "https://www.metacritic.com/browse/movie/all/all/2020/metascore/?page=1"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

# for number of pages required to search through
pagenum = soup.find_all(class_ = "c-navigationPagination_itemButtonContent")
pagenum = ((pagenum[3].text).split("\n")[1])[12:]

AllPages = []


# running the code the number of page numbers
for page in range(int(pagenum)):
  newURL = "https://www.metacritic.com/browse/movie/all/all/2020/metascore/?page=" + str(page+1)

  response = requests.get(newURL, headers=headers)

  from bs4 import BeautifulSoup

  soup = BeautifulSoup(response.text, 'html.parser')
  productListings = soup.find_all(class_='c-productListings')
  productListings;

  title_element = soup.find_all(class_='c-finderProductCard_titleHeading')
  date_element = soup.find_all(class_='u-text-uppercase')
  summary_element = soup.find_all(class_='c-finderProductCard_description')
  rating_element = soup.find_all(class_='c-finderProductCard_meta')
  score_element = soup.find_all(class_='c-finderProductCard_score')

  titles = []
  for title in title_element:
    titles.append(title.text.split(" ", 1)[1])

  dates = []
  for date in date_element:
    dates.append(date.text[11:].split('\n')[0])

  summaries = []
  for summary in summary_element:
    summaries.append(summary.text)

  ratings = []
  count = 0
  for rating in rating_element:
    if count % 2 == 0:
      try:
        ratings.append((rating.text.split('\n')[4])[15:])
      except:
        ratings.append('Not Rated')
    count += 1

  scores = []
  for score in score_element:
    scores.append(score.text[:2])

  data = {
      'Title': titles,
      'Date': dates,
      'Summary': summaries,
      'Rating': ratings,
      'Metascore': scores
  }

  #making a dataframe for all the data to be inserted
  df = pd.DataFrame(data)
  AllPages.append(df)

AllPages = pd.concat(AllPages).reset_index(drop=True)
AllPages.index += 1


In [273]:
AllPages

,Title,Date,Summary,Rating,Metascore
1,Small Axe: Lovers Rock,"Nov 27, 2020",A single evening at a house party in 1980s Wes...,TV-MA,95
2,Collective,"Nov 20, 2020",Collective follows a heroic team of journalist...,Not Rated,95
3,David Byrne’s American Utopia,"Oct 17, 2020",David Byrne’s American Utopia brings the criti...,TV-14,93
4,Never Rarely Sometimes Always,"Mar 13, 2020",Faced with an unintended pregnancy and a lack ...,PG-13,92
5,Time,"Oct 9, 2020","Fox Rich is a fighter. The entrepreneur, aboli...",PG-13,91
...,...,...,...,...,...
539,The Tax Collector,"Aug 7, 2020","David (Bobby Soto) and Creeper (Shia LaBeouf),...",TV-MA,22
540,Blumhouse's Fantasy Island,"Feb 14, 2020","In Blumhouse's new spin on Fantasy Island, the...",PG-13,22
541,The Last Days of American Crime,"Jun 5, 2020","As a final response to terrorism and crime, th...",TV-MA,15
542,"Love, Weddings & Other Disasters","Dec 4, 2020",With a couple’s impending wedding as the backd...,PG-13,11


In [274]:
AllPages.to_csv("DAC_Metacritic_Project.csv")